### **Librerías**

In [54]:
import os
import math
import time
import smtplib

import pyautogui
import threading
import pywhatkit

import numpy as np
import pandas as pd

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

### **BigQuery**

In [64]:
# Proyecto y cliente
project_id='bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Configuración job
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

query = """
    SELECT DISTINCT
        dateCreatedAt,
        sellerId,
        sellerName,
        segmento,
        deliveryOrderNumber,
        sellerPhone,
        sellerEmail,
        clientDNI,
        clientName,
        clientPhone,
        clientEmail,
        shipping_fee,
        TotalPrecio,
        monto_total,
        department,
        province,
        district,
        tipo_documento,
        razon_social,
        direccion,
        region,
        provincia,
        persona_fiscal,
        tipo_de_documento,
        identificacion_legal,
        actividad_economica,
        digito_verificador,
        distrito,
        correo_electronico,
        telefono,
        codigo_postal,
        invoiceDate,
        invoiceType,
        invoiceNumber,
        fecha_solicitud,
        ticket_solicitud,
        caseTipification,
        caseStatus,
        fecha_inicio_solicitud,
        fecha_final_solicitud,
        status_comprobante,
        dias_comprobante,
        dias_solicitud,
        n_solicitudes,
        loyaltyPhone,
        EnlaceAAA
    FROM (
        SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_comprobantes` 
        WHERE invoiceDate IS NULL
        AND DATE_DIFF(CURRENT_DATE(),datecreatedAt,DAY)<=15
        AND dias_comprobante>0
        AND integrado='No'
        UNION ALL
        SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_comprobantes` 
        WHERE ticket_solicitud IS NOT NULL
        AND caseStatus!='Closed'
        AND status_comprobante!='Enviado'
    )
    ORDER BY dateCreatedAt DESC
"""

# Query execution
query_job = client.query(query, job_config=job_config)
query_job.result()
data = query_job.to_dataframe()

c:\Users\gfloress\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### **Variables**

In [56]:
# Correo

correo_usuario = 'sleona@falabella.com'
contrasena = 'I$H#pm3457'
asuntos_base = [
    'F.COM | 🚨 ENVÍA TUS COMPROBANTES PENDIENTES 🚨'
]

# Columnas

columnas = [
    'sellerName',
    'dateCreatedAt','deliveryOrderNumber', 
    'shipping_fee','TotalPrecio','monto_total',
    'tipo_documento', 
    'tipo_de_documento', 'identificacion_legal',
    'razon_social', 
    'region', 'provincia', 'distrito', 'direccion',
    #'correo_electronico', 'telefono',  
    #'persona_fiscal',  
    #'actividad_economica', 'digito_verificador', 
    #'codigo_postal'
]

# Variables iniciales

# NOMBRE, DIRECCION, DNI, FECHA DE EMISION, 

In [57]:
data.columns

Index(['dateCreatedAt', 'sellerId', 'sellerName', 'segmento',
       'deliveryOrderNumber', 'sellerPhone', 'sellerEmail', 'clientDNI',
       'clientName', 'clientPhone', 'clientEmail', 'shipping_fee',
       'TotalPrecio', 'monto_total', 'tipo_documento', 'razon_social',
       'direccion', 'region', 'provincia', 'persona_fiscal',
       'tipo_de_documento', 'identificacion_legal', 'actividad_economica',
       'digito_verificador', 'distrito', 'correo_electronico', 'telefono',
       'codigo_postal', 'invoiceDate', 'invoiceType', 'invoiceNumber',
       'fecha_solicitud', 'ticket_solicitud', 'caseTipification', 'caseStatus',
       'fecha_inicio_solicitud', 'fecha_final_solicitud', 'status_comprobante',
       'dias_comprobante', 'dias_solicitud', 'n_solicitudes', 'loyaltyPhone',
       'EnlaceAAA'],
      dtype='object')

In [58]:
data[columnas].query("tipo_documento=='Factura'") #shipping_fee
# la factura tiene que tener el shipping_fee + paid_price esto debe ser el monto_total

,sellerName,dateCreatedAt,deliveryOrderNumber,shipping_fee,TotalPrecio,monto_total,tipo_documento,tipo_de_documento,identificacion_legal,razon_social,region,provincia,distrito,direccion
1,COMERCIAL RRJ E.I.R.L,2024-06-05,2241322752,89.0,999.000000000,1088.00,Factura,RUC,20132707597,FANTASIA SAC,Piura,Piura,Piura,JR.LOS NARANJOS 272 URB.SANTA ISABEL
7,ELECTROHOGAR,2024-06-05,2240706214,89.0,1249.000000000,1338.00,Factura,RUC,20612409821,fundamenta proyectos y contrucciones srl,Lima,Lima,San Isidro,paseo de la republica
13,BUYPAL STORE PERU,2024-06-05,2241238643,0.0,29.000000000,29.00,Factura,RUC,20328369096,Comunidad Cristiana de Arequipa,Arequipa,Arequipa,Arequipa,Peral 609
35,Importadora Esn Cursor,2024-06-05,2243400170,0.0,159.600000000,159.60,Factura,RUC,20602560539,Multiservicios Marshey SAC,PIURA,PIURA,CASTILLA,Av. Monteerde Maanzana P Lote 13 Etapa II
52,Ofertec,2024-06-05,2237812810,19.0,89.000000000,108.00,Factura,RUC,20608267124,CORPORACION NOVATEC SAC,Ayacucho,Huanta,Huanta,AV. MARISCAL CASTILLA NRO. 459 AYACUCHO - HUAN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82820,Sany Distribuidor Autorizado,2024-04-09,2191379679,0.0,849.000000000,849.00,Factura,RUC,20601695503,negu telecominiciones perú E. I. R. L,JUNÍN,JAUJA,JAUJA,jr 28 de julio 218 jauja
82821,MCR SUMINISTROS SAC,2024-04-09,2191448013,0.0,79.950000000,79.95,Factura,RUC,20506629188,Gerencia de Proyecto y Construccion SAC,Lima,Lima,San Isidro,Calle Las Orquideas 585 Of. 301a
82822,Nibray,2024-04-06,2189909915,0.0,239.800000000,239.80,Factura,RUC,10444195865,Jack Andrés Beltrán torres,UCAYALI,CORONEL PORTILLO,CALLERIA,Calle contamana 128
82823,Visto de los Andes,2024-04-06,2189909916,0.0,65.000000000,65.00,Factura,RUC,10444195865,Jack Andrés Beltrán torres,UCAYALI,CORONEL PORTILLO,CALLERIA,Calle contamana 128


### **Funciones**

In [59]:
"""
def send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):
    # Set up the SMTP server
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()

    # Log in to the SMTP server
    server.login(correo_usuario, contrasena)

    # Set up the message
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Attach the HTML message
    msg.attach(MIMEText(html_message, 'html'))

    # Send the email
    server.sendmail(sender_email, seller_email, msg.as_string())

    # Close the SMTP connection
    server.quit()
"""

"\ndef send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):\n    # Set up the SMTP server\n    server = smtplib.SMTP('smtp.office365.com', 587)\n    server.starttls()\n\n    # Log in to the SMTP server\n    server.login(correo_usuario, contrasena)\n\n    # Set up the message\n    msg = MIMEMultipart()\n    msg['From'] = 'fcomlogisticasx@falabella.com'\n    msg['To'] = correo_seller\n    msg['Subject'] = asunto\n    msg['CC'] = 'soporteperufcom@falabella.com'\n\n    # Attach the HTML message\n    msg.attach(MIMEText(html_message, 'html'))\n\n    # Send the email\n    server.sendmail(sender_email, seller_email, msg.as_string())\n\n    # Close the SMTP connection\n    server.quit()\n"

### **Mensajes**

### **Data**

In [62]:
data.columns

Index(['dateCreatedAt', 'sellerId', 'sellerName', 'segmento',
       'deliveryOrderNumber', 'sellerPhone', 'sellerEmail', 'clientDNI',
       'clientName', 'clientPhone', 'clientEmail', 'shipping_fee',
       'TotalPrecio', 'monto_total', 'tipo_documento', 'razon_social',
       'direccion', 'region', 'provincia', 'persona_fiscal',
       'tipo_de_documento', 'identificacion_legal', 'actividad_economica',
       'digito_verificador', 'distrito', 'correo_electronico', 'telefono',
       'codigo_postal', 'invoiceDate', 'invoiceType', 'invoiceNumber',
       'fecha_solicitud', 'ticket_solicitud', 'caseTipification', 'caseStatus',
       'fecha_inicio_solicitud', 'fecha_final_solicitud', 'status_comprobante',
       'dias_comprobante', 'dias_solicitud', 'n_solicitudes', 'loyaltyPhone',
       'EnlaceAAA'],
      dtype='object')

In [63]:
data[columnas]

,sellerName,dateCreatedAt,deliveryOrderNumber,shipping_fee,TotalPrecio,monto_total,tipo_documento,tipo_de_documento,identificacion_legal,razon_social,region,provincia,distrito,direccion
0,VELOCELL,2024-06-05,2241149292,0.0,649.000000000,649.00,Boleta,None,None,None,None,None,None,None
1,COMERCIAL RRJ E.I.R.L,2024-06-05,2241322752,89.0,999.000000000,1088.00,Factura,RUC,20132707597,FANTASIA SAC,Piura,Piura,Piura,JR.LOS NARANJOS 272 URB.SANTA ISABEL
2,Sany Distribuidor Autorizado,2024-06-05,2240509093,0.0,1399.000000000,1399.00,Boleta,None,None,None,None,None,None,None
3,Importadora Esn Cursor,2024-06-05,2243340414,0.0,30.900000000,30.90,Boleta,None,None,None,None,None,None,None
4,TAPOUT,2024-06-05,2240867017,0.0,39.900000000,39.90,Boleta,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82854,RODALEXIA,2023-05-11,2062427666,6.5,1199.000000000,1205.50,Boleta,None,None,None,None,None,None,None
82855,ZARCEX NETWORK,2023-05-04,2061033267,12.0,639.990000000,651.99,Boleta,None,None,None,None,None,None,None
82856,N&R Telcom,2023-04-17,2057744847,0.0,1379.000000000,1379.00,Boleta,None,None,None,None,None,None,None
82857,Imaco Tienda Oficial,2023-03-16,2052933519,0.0,339.000000000,339.00,Boleta,None,None,None,None,None,None,None


In [61]:
# Leer el registro desde un archivo Excel
data = (
        data
        .query("sellerEmail.str.contains('@')")
        .assign(**{
            'shipping_fee': lambda x: round(pd.to_numeric(x['shipping_fee']),2),
            'TotalPrecio': lambda x: round(pd.to_numeric(x['TotalPrecio']),2),
            'monto_total': lambda x: round(pd.to_numeric(x['monto_total']),2),
            'tipo_de_documento': lambda x: np.where(x['tipo_documento']=='Boleta','DNI',x['tipo_documento']),
            'identificacion_legal': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientDNI'],x['identificacion_legal']),
            'razon_social': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientName'],x['razon_social']),
            'region': lambda x: np.where(x['tipo_documento']=='Boleta','DNI',x['department']),
            'provincia': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientDNI'],x['province']),
            'distrito': lambda x: np.where(x['tipo_documento']=='Boleta',x['clientName'],x['district'])
        })
        .loc[:,columnas]
)

KeyError: 'department'

### **Condicionales**

In [53]:
data[columnas].query("tipo_documento=='Boleta'")

,sellerName,dateCreatedAt,deliveryOrderNumber,shipping_fee,TotalPrecio,monto_total,tipo_documento,tipo_de_documento,identificacion_legal,razon_social,region,provincia,distrito,direccion
0,SPORT LIFE PERU,2024-06-05,2240246591,9.9,298.00,307.90,Boleta,DNI,40914475,Moised Pacheco,None,None,None,None
1,MEGACONSTRUCTORES,2024-06-05,2241001048,0.0,299.00,299.00,Boleta,DNI,44254850,Marti Garcia Marti Garcia,None,None,None,None
2,Lets Play,2024-06-05,2241880120,8.9,139.00,147.90,Boleta,DNI,75712442,Shery Aguilar,None,None,None,None
3,PERFUMES PREMIUM PERU,2024-06-05,2241185926,19.0,299.90,318.90,Boleta,DNI,45056370,Maria Del Rosario Salas,None,None,None,None
4,SPORT LIFE PERU,2024-06-05,2240561963,19.0,149.00,168.00,Boleta,DNI,73123804,Jose Vega,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82854,RODALEXIA,2023-05-11,2062427666,6.5,1199.00,1205.50,Boleta,DNI,46462141,Rosario Nazareth Villanueva,None,None,None,None
82855,ZARCEX NETWORK,2023-05-04,2061033267,12.0,639.99,651.99,Boleta,DNI,43576468,Vidal Condori,None,None,None,None
82856,N&R Telcom,2023-04-17,2057744847,0.0,1379.00,1379.00,Boleta,DNI,43618462,Carlos Oswaldo Manjo,None,None,None,None
82857,Imaco Tienda Oficial,2023-03-16,2052933519,0.0,339.00,339.00,Boleta,DNI,45552539,Deyanira Jaramillo,None,None,None,None


In [237]:
# Correos


In [7]:
(                 
data[columnas]
                    .drop_duplicates()
                    #.sort_values(by='Estatusenvio', ascending=False)
                    #.rename(columns={
                    #          'sellerName': 'Nombre del seller',
                    #          'deliveryOrderNumber': 'Número de orden',
                    #          'trackingCode': 'Rastreo',
                    #          'description': 'Descripción',
                    #          'promisedBySeller': 'Fecha de promesa',
                    #          'Estatusenvio': 'Estado del envío'
                    #})
)

#para boleta: 

# - nombre cliente (razon_social)
# - dni (tipo_documento | identificacion_legal)


,sellerName,dateCreatedAt,deliveryOrderNumber,monto_total,tipo_documento,razon_social,direccion,region,provincia,persona_fiscal,tipo_de_documento,identificacion_legal,actividad_economica,digito_verificador,distrito,correo_electronico,telefono,codigo_postal
0,ELECTROHOGAR,2024-06-05,2240706214,1338.00,Factura,fundamenta proyectos y contrucciones srl,paseo de la republica,Lima,Lima,business,RUC,20612409821,4100 - CONSTRUCCIÓN DE EDIFICIOS,1,San Isidro,pattymuj@gmail.com,+51957447063,-
1,SPORT LIFE PERU,2024-06-05,2240561963,168.00,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Importadora Esn Cursor,2024-06-05,2240811234,61.80,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
3,BESTOFFERSPERU,2024-06-05,2240990695,27.90,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
4,JPSYSTEMS,2024-06-05,2240449919,82.10,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82854,RODALEXIA,2023-05-11,2062427666,1205.50,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
82855,ZARCEX NETWORK,2023-05-04,2061033267,651.99,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
82856,N&R Telcom,2023-04-17,2057744847,1379.00,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None
82857,Imaco Tienda Oficial,2023-03-16,2052933519,339.00,Boleta,None,None,None,None,None,None,None,None,None,None,None,None,None


### **Envío de correos**

#### **1. AAA**

In [238]:
# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data['sellerId'].unique():
    
    # Filtrar dataframe por tienda
    df_seller = data[data['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller =  (
                    df_seller[columnas]
                    .drop_duplicates()
                    .sort_values(by='Estatusenvio', ascending=False)
                    .rename(columns={
                              'sellerName': 'Nombre del seller',
                              'deliveryOrderNumber': 'Número de orden',
                              'trackingCode': 'Rastreo',
                              'description': 'Descripción',
                              'promisedBySeller': 'Fecha de promesa',
                              'Estatusenvio': 'Estado del envío'
                    })
                  )
    
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[0].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: #74B700;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}!</strong> Recuerda subir los comprobantes de los siguientes pedidos: </h2></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>
          <p>Para cada orden debes subir la boleta o factura emitida a Falabella Seller Center.</p>
              <ol>
                  <li>Revisa nuestro tutorial de facturación: <a href="https://ayudaseller.falabella.com/s/article/Generacion-de-boletas-y-facturas-Peru">Tutorial de facturación</a>.</li>
                  <li>Revisa nuestro tutorial de carga de documentos: <a href="https://ayudaseller.falabella.com/s/article/Carga-de-documentos-tributarios">Tutorial de carga de comprobantes</a>.</li>
              </ol>
                            
          <p>🚨 Este paso es obligatorio 🚨 de no hacerlo podrías caer en penalidades.</p>
          <p>🚨 Recuerda que los datos de empresa deben coincidir con tus datos de Falabella Seller Center 🚨.</p>

          <p>Podrás revisar tus órdenes con comprobante pendiente en Falabella Seller Center desde <a href= "https://sellercenter.falabella.com/order/invoice#/purchased-order-list">aquí</a>.</p>

          <p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
          
          <p>¡Gracias!</p> 
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, correo_seller, msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

### **Envío de Whatsapp**

#### **1. AAA**

In [244]:
def send_whatsapp_message(seller_data):
    mensaje_whatsapp = """
        *Hola {sellerName}, Hoy estás en la ruta de recolección*

        Detectamos que tienes órdenes poseen retraso en la entrega, es importante que las entregues hoy al operador logístico.

        Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

        🚚 Aquí podrás revisar el tutorial de gestión de órdenes: https://youtu.be/lpk1_y9uYDg

        Queremos reforzar contigo el proceso para gestionarlas:

        📦 Para que podamos recolectar tu orden deberás:

        1-	Podrás revisar tus órdenes en falabella seller center desde aquí: https://sellercenter.falabella.com/order
        2-	Recuerda hacer clic en “Listo para despachar” como máximo un día antes de la fecha límite de despacho (antes de las 10p.m.)

        🌟Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el *{Phone}* o haz clic aquí {link_phone}

        ¡Gracias!
    """
    whatsapp_message = mensaje_whatsapp.format(
        sellerName=seller_data['sellerName'],
        correo=seller_data['sellerEmail'],
        Phone=seller_data['sellerPhone'],
        link_phone=seller_data['loyaltyPhone'],
    )
    
    pywhatkit.sendwhatmsg_instantly(seller_data['sellerPhone'], whatsapp_message)

In [245]:
lista = data_5.groupby(['sellerId'],dropna=False).agg(orders=('deliveryOrderNumber','count')).reset_index().sort_values(['orders'], ascending=False)
sellers_lista = lista[lista['orders']>=5]['sellerId'].to_list()
data_5_final = data_5[data_5['sellerId'].isin(sellers_lista)]

In [246]:
data_5_final['sellerId'].drop_duplicates().count()

21

In [247]:
# Batch processing
threads = []
for seller_id, seller_data in data_5_final.groupby('sellerId'):
    seller_data = seller_data.iloc[0]  # Assuming each seller has the same data
    thread = threading.Thread(target=send_whatsapp_message, args=(seller_data,))
    threads.append(thread)
    thread.start()
    time.sleep(12)  # Add delay between starting threads

# Wait for all threads to finish
for thread in threads:
    thread.join()